In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.read_excel("BTC-USD.xlsx").head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800.0
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200.0
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700.0
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600.0
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100.0


In [3]:
BTC = pd.read_excel("BTC-USD.xlsx")

In [4]:
BTC_modify = pd.DataFrame(BTC).dropna()

In [5]:
import numpy
import talib

close = numpy.random.random(30)
print(type(close))
print(talib.SMA(close, timeperiod=5))

close = BTC_modify["Close"].to_numpy()
print(type(close))
print(talib.SMA(close, timeperiod=5))

<class 'numpy.ndarray'>
[       nan        nan        nan        nan 0.48967023 0.37597598
 0.19626316 0.2842878  0.22827715 0.3085649  0.34743234 0.48471154
 0.51433257 0.64677655 0.57489134 0.65353809 0.62588948 0.56684204
 0.51598902 0.49713925 0.51262646 0.41899722 0.4754693  0.41001602
 0.42180828 0.37746812 0.55203151 0.48297896 0.53573694 0.50537139]
<class 'numpy.ndarray'>
[          nan           nan           nan ... 43333.1203126 42137.6304688
 40795.0625   ]


In [6]:
# note that all ndarrays must be the same length!
inputs = {
    'open': BTC_modify["Open"].to_numpy(),
    'high': BTC_modify["High"].to_numpy(),
    'low': BTC_modify["Low"].to_numpy(),
    'close': BTC_modify["Close"].to_numpy(),
    'volume': BTC_modify["Volume"].to_numpy()
}

from talib.abstract import *

SMA(inputs, timeperiod=5) # calculate on close prices by default
# print(output)
# print(len(list(output)))
# print(pd.DataFrame(output).dropna())
# print(pd.DataFrame(output).dropna().shape)

array([          nan,           nan,           nan, ..., 43333.1203126,
       42137.6304688, 40795.0625   ])

In [7]:
BTC_modify["5ma"]=SMA(inputs,timeperiod=5)

In [8]:
BTC_modify["3ma"]=None
BTC_modify["6ma"]=None
BTC_modify["12ma"]=None
BTC_modify["3ma"]=SMA(inputs,timeperiod=3)
BTC_modify["6ma"]=SMA(inputs,timeperiod=6)
BTC_modify["12ma"]=SMA(inputs,timeperiod=12)

In [9]:
BTC_input=pd.DataFrame(BTC_modify).dropna()

In [10]:
BTC_input["buy_day"]=0
BTC_input["cum_day"]=0
BTC_input["sell_day"]=0
BTC_input["number"]=0
BTC_input["value"]=0
BTC_input["period_return"]=0
BTC_input["holding_period"]=0
BTC_input["annual_return"]=0
BTC_input["diff"]=0


In [11]:
#Strategy of trading
#BUT: 3MA>7MA or 3M>12MA
#SELL: 7MA<12MA
# OR HOLDING

In [12]:
for i in range(1, BTC_input.shape[0]):
    if((BTC_input["3ma"].iloc[i] > BTC_input["6ma"].iloc[i])\
    or(BTC_input["3ma"].iloc[i] > BTC_input["12ma"].iloc[i]))\
    and(BTC_input["cum_day"].iloc[i-1]==0):
        BTC_input["buy_day"].iloc[i]  = 1
        BTC_input["cum_day"].iloc[i]  = 1
        BTC_input["sell_day"].iloc[i] = 0
    elif(BTC_input["12ma"].iloc[i] < BTC_input["6ma"].iloc[i])and(BTC_input["cum_day"].iloc[i-1]==1):
        BTC_input["buy_day"].iloc[i]  = 0
        BTC_input["cum_day"].iloc[i]  = 0
        BTC_input["sell_day"].iloc[i] = 1
    elif(BTC_input["cum_day"].iloc[i-1]>0):
        BTC_input["buy_day"].iloc[i]  = 0
        BTC_input["cum_day"].iloc[i]  = 1
        BTC_input["sell_day"].iloc[i] = 0

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [13]:
BTC_input.to_excel("progb.xlsx")

In [14]:
#add
initial=10000
for i in range(1, BTC_input.shape[0]):
#buy signal
    if(BTC_input["buy_day"].iloc[i]==1)and(BTC_input["cum_day"].iloc[i]==1)and(BTC_input["sell_day"].iloc[i]==0):
        BTC_input["number"].iloc[i] =initial/BTC_input["Adj Close"].iloc[i]
        BTC_input["value"].iloc[i]=BTC_input["number"].iloc[i]*BTC_input["Adj Close"].iloc[i]
        BTC_input["period_return"].iloc[i]=round((BTC_input["number"].iloc[i]*BTC_input["Adj Close"].iloc[i]-initial)/initial,4)
#sell signal
    elif(BTC_input["buy_day"].iloc[i]==0)and(BTC_input["cum_day"].iloc[i]==0)and(BTC_input["sell_day"].iloc[i]==1):
        BTC_input["number"].iloc[i]=0
        BTC_input["value"].iloc[i]=BTC_input["number"].iloc[i-1]*BTC_input["Adj Close"].iloc[i]
        BTC_input["period_return"].iloc[i]=round((BTC_input["number"].iloc[i-1]*BTC_input["Adj Close"].iloc[i]-initial)/initial,4)
#hoding
    elif(BTC_input["buy_day"].iloc[i]==0)and(BTC_input["cum_day"].iloc[i]==1)and(BTC_input["sell_day"].iloc[i]==0):
        BTC_input["number"].iloc[i]=BTC_input["number"].iloc[i-1]
        BTC_input["value"].iloc[i]=BTC_input["number"].iloc[i]*BTC_input["Adj Close"].iloc[i]
        BTC_input["period_return"].iloc[i]=round((BTC_input["number"].iloc[i]*BTC_input["Adj Close"].iloc[i]-initial)/initial,4)

In [15]:
BTC_input.to_excel("progb0604.xlsx")

In [16]:
start = 0
end = 0
for i in range(1, len(BTC_input)):
    if(BTC_input["buy_day"].iloc[i]==1):
        BTC_input["holding_period"].iloc[i] = 1
    elif((BTC_input["cum_day"].iloc[i] > 0) or (BTC_input["sell_day"].iloc[i] == 1))and(BTC_input["buy_day"].iloc[i] == 0):
        BTC_input["holding_period"].iloc[i] = BTC_input["holding_period"].iloc[i-1] +1
BTC_input.head(6)

,Date,Open,High,Low,Close,Adj Close,Volume,5ma,3ma,6ma,12ma,buy_day,cum_day,sell_day,number,value,period_return,holding_period,annual_return,diff
11,2014-09-28,399.471008,401.016998,374.332001,377.181000,377.181000,23613300.0,403.180994,393.708659,408.615992,411.511915,0,0,0,0.0,0.0,0.0,0,0,0
12,2014-09-29,376.928009,385.210999,372.239990,375.467010,375.467010,32497700.0,393.633398,384.056000,398.561996,404.689664,0,0,0,0.0,0.0,0.0,0,0,0
13,2014-09-30,376.088013,390.976990,373.442993,386.944000,386.944000,34707300.0,388.707397,379.864003,392.518499,401.564997,0,0,0,0.0,0.0,0.0,0,0,0
14,2014-10-01,387.427002,391.378998,380.779999,383.614990,383.614990,26229400.0,384.545398,382.008667,387.858663,400.633247,0,0,0,0.0,0.0,0.0,0,0,0
15,2014-10-02,383.988007,385.497009,372.946014,375.071991,375.071991,21777700.0,379.655798,381.876994,382.966497,397.813914,0,0,0,0.0,0.0,0.0,0,0,0
16,2014-10-03,375.181000,377.695007,357.859009,359.511993,359.511993,30901200.0,376.121997,372.732991,376.298497,394.538162,0,0,0,0.0,0.0,0.0,0,0,0


In [17]:
for i in range(1, len(BTC_input)):
    if(BTC_input["buy_day"].iloc[i]==1)or(BTC_input["sell_day"].iloc[i]==1)or(BTC_input["cum_day"].iloc[i]==1):
        BTC_input["annual_return"].iloc[i]= (1+BTC_input["period_return"].iloc[i])**(365/BTC_input["holding_period"].iloc[i])-1

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
BTC_input.head(25)

,Date,Open,High,Low,Close,Adj Close,Volume,5ma,3ma,6ma,12ma,buy_day,cum_day,sell_day,number,value,period_return,holding_period,annual_return,diff
11,2014-09-28,399.471008,401.016998,374.332001,377.181000,377.181000,23613300.0,403.180994,393.708659,408.615992,411.511915,0,0,0,0.000000,0.000000,0.0000,0,0.000000,0
12,2014-09-29,376.928009,385.210999,372.239990,375.467010,375.467010,32497700.0,393.633398,384.056000,398.561996,404.689664,0,0,0,0.000000,0.000000,0.0000,0,0.000000,0
13,2014-09-30,376.088013,390.976990,373.442993,386.944000,386.944000,34707300.0,388.707397,379.864003,392.518499,401.564997,0,0,0,0.000000,0.000000,0.0000,0,0.000000,0
14,2014-10-01,387.427002,391.378998,380.779999,383.614990,383.614990,26229400.0,384.545398,382.008667,387.858663,400.633247,0,0,0,0.000000,0.000000,0.0000,0,0.000000,0
15,2014-10-02,383.988007,385.497009,372.946014,375.071991,375.071991,21777700.0,379.655798,381.876994,382.966497,397.813914,0,0,0,0.000000,0.000000,0.0000,0,0.000000,0
16,2014-10-03,375.181000,377.695007,357.859009,359.511993,359.511993,30901200.0,376.121997,372.732991,376.298497,394.538162,0,0,0,0.000000,0.000000,0.0000,0,0.000000,0
17,2014-10-04,359.891998,364.487000,325.885986,328.865997,328.865997,47236500.0,366.801794,354.483327,368.245997,388.430995,0,0,0,0.000000,0.000000,0.0000,0,0.000000,0
18,2014-10-05,328.915985,341.800995,289.295990,320.510010,320.510010,83308096.0,353.514996,336.296000,359.086497,378.824247,0,0,0,0.000000,0.000000,0.0000,0,0.000000,0
19,2014-10-06,320.389008,345.134003,302.559998,330.079010,330.079010,79011800.0,342.807800,326.485006,349.608998,371.063749,0,0,0,0.000000,0.000000,0.0000,0,0.000000,0
20,2014-10-07,330.584015,339.247009,320.481995,336.187012,336.187012,49199900.0,335.030804,328.925344,341.704335,364.781499,0,0,0,0.000000,0.000000,0.0000,0,0.000000,0


In [19]:
annual_return=1
annual_period_return=1
trading=0

for i in range(0,BTC_input.shape[0]):
    if(BTC_input["sell_day"].iloc[i]==1):
        annual_return *=(1+BTC_input["annual_return"].iloc[i])
        annual_period_return *=(1+BTC_input["period_return"].iloc[i])
        trading+=1

final_return=annual_return**(1/trading)-1
final_period_return=annual_period_return**(1/trading)-1
print("交易次數：",trading,"年化報酬",final_return,"期間年化報酬",final_period_return)

交易次數： 687 年化報酬 1.61945717175259 期間年化報酬 0.005138921451674383


In [20]:
annual_return = 1
annual_period_return = 1
trading = 0

for i in range(0, BTC_input.shape[0]):
    if(BTC_input["sell_day"].iloc[i]==1):
        annual_return *= (1 + BTC_input["annual_return"].iloc[i] )
        annual_period_return *= (1 + BTC_input["period_return"].iloc[i] )
        trading+=1

final_return=annual_return**(1/trading)-1
final_period_return=annual_period_return**(1/trading)-1
print("交易次數:", trading, "年化報酬:", final_return, "期間年化報酬:", final_period_return)

交易次數: 687 年化報酬: 1.61945717175259 期間年化報酬: 0.005138921451674383


In [21]:
BTC_input.to_excel("AT_Final.xlsx")